### Model Composition ServerHandle APIs

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_04_inference_graphs.ipynb) <br>
⬅️ [Previous notebook](./ex_02_ray_serve_fastapi.ipynb) <br>

<img src="images/PatternsMLProduction.png" width="70%" height="40%">

### Learning Objective:
In this tutorial, you will learn how to:

 * compose models together as nodes of an inference graph. 
 * scale each model independently as part of the graph by increasing replicas 
 * use a single class to encompass all models as nodes into a deployment graph


In this short tutorial we going to use HuggingFace Transformer 🤗 to accomplish three tasks:
 1. Analyse the sentiment of a tweet: Positive or Negative
 2. Translate it into French
 3. Demonstrate the model composition deployment pattern using an inference graph
 
 <img src="images/sentiment_analysis.jpeg" width="70%" height="40%">

In [ ]:
from transformers import TranslationPipeline, TextClassificationPipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
import torch
import requests
from ray import serve

These are example 🐦 tweets, some made up, some extracted from a dog lover's twitter handle. In a real use case,
these could come live from a Tweeter handle using [Twitter APIs](https://developer.twitter.com/en/docs/twitter-api/getting-started/getting-access-to-the-twitter-api). 

In [ ]:
TWEETS = ["Tonight on my walk, I got mad because mom wouldn't let me play with this dog. We stared at each other...he never blinked!",
          "Sometimes. when i am bored. i will stare at nothing. and try to convince the human. that there is a ghost",
          "You little dog shit, you peed and pooed on my new carpet. Bad dog!",
          "I would completely believe you. Dogs and little children - very innocent and open to seeing such things",
          "You've got too much time on your paws. Go check on the skittle. under the, fridge",
          "You sneaky little devil, I can't live without you!!!",
          "It's true what they say about dogs: they are you BEST BUDDY, no matter what!",
          "This dog is way dope, just can't enough of her",
          "This dog is way cool, just can't enough of her",
          "Is a dog really the best pet?",
          "Cats are better than dogs",
          "Totally dissastified with the dog. Worst dog ever",
          "Briliant dog! Reads my moods like a book. Senses my moods and reacts. What a companinon!"
          ]

Utiliy function to fetch a tweet; these could very well be live tweets coming from Twitter API for a user or a #hashtag

In [ ]:
def fetch_tweet_text(i):
    text = TWEETS[i]
    return text

### Sentiment model deployment

Our function deployment model to analyse the tweet using a pretrained transformer from HuggingFace 🤗.
Note we have number of `replicas=1` but to scale it, we can increase the number of replicas, as
we have done below.

In [ ]:
@serve.deployment(num_replicas=1)
class SentimentTweet:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", model_max_length=128)
        self.model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
        self.pipeline = TextClassificationPipeline(model=self.model, tokenizer=self.tokenizer, task="sentiment-analysis")

    def sentiment(self, text: str):
        return (f"label:{self.pipeline(text)[0]['label']}; score={self.pipeline(text)[0]['score']}")

### Translation model deployment

Our function to translate a tweet from English --> French using a pretrained Transformer from HuggingFace 🤗

In [ ]:
@serve.deployment(num_replicas=2)
class TranslateTweet:
    def __init__(self):
         self.tokenizer = AutoTokenizer.from_pretrained("t5-small", model_max_length=128)
         self.model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
         self.use_gpu = 0 if torch.cuda.is_available() else -1
         self.pipeline = TranslationPipeline(self.model, self.tokenizer, task="translation_en_to_fr", device=self.use_gpu)

    def translate(self, text: str):
        return self.pipeline(text)[0]['translation_text']

### Use the Model Composition pattern

<img src="images/tweet_composition.png" width="60%" height="25%">

A composed class is deployed with both sentiment analysis and translations models' ServeHandles initialized in the constructor.

In [ ]:
@serve.deployment(route_prefix="/composed", num_replicas=2)
class ComposedModel:
    def __init__(self, translate, sentiment):
        # fetch and initialize deployment handles
        self.translate_model = translate
        self.sentiment_model = sentiment

    async def __call__(self, http_request):
        data = await http_request.json()
        sentiment_ref = await self.sentiment_model.sentiment.remote(data)
        trans_text_ref = await self.translate_model.translate.remote(data)
        sentiment_val = await sentiment_ref
        trans_text = await trans_text_ref

        return {'Sentiment': sentiment_val, 'Translated Text': trans_text}

### Deploy our models 

Deploy our models. Our composed class is deployed with both sentiment analysis and translations models' `ClassNode` initialized in the constructor.

In [ ]:
translate_cls_node = TranslateTweet.bind()
sentiment_cls_node = SentimentTweet.bind()
compose_cls_node = ComposedModel.bind(translate_cls_node,sentiment_cls_node )

serve.run(compose_cls_node)

### Send HTTP requests to our deployment model

In [ ]:
for i in range(len(TWEETS)):
    tweet = fetch_tweet_text(i)
    response = requests.post("http://127.0.0.1:8000/composed", json=tweet)
    print(f"tweet request... : {tweet}")
    print(f"tweet response:{response.text}")

Gracefully shutdown the Ray serve instance.

In [ ]:
serve.shutdown()

### Exercise

1. Add more tweets to `TWEETS` with different sentiments.
2. Check the score (and if you speak and read French, what you think of the translation?)

### Homework

1. Instead of French, use a language transformer of your choice
2. What about Neutral tweets? Try using [vaderSentiment](https://github.com/cjhutto/vaderSentiment)
3. Solution for 2) is [here](https://github.com/anyscale/academy/blob/main/ray-serve/05-Ray-Serve-SentimentAnalysis.ipynb)

### Next

We'll further explore model composition using [Deploymant Graph APIs](https://docs.ray.io/en/latest/serve/deployment-graph.html).

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_04_inference_graphs.ipynb) <br>
⬅️ [Previous notebook](./ex_02_ray_serve_fastapi.ipynb) <br>